# Main ML code below

In [1]:
import numpy as np

from IPython.display import Image
from IPython.display import Markdown

import matplotlib
import matplotlib.pyplot as plt

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import torchvision.models as models
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

import torch.optim as optim

from torch.autograd import Variable

In [2]:
# License: BSD
# Author: Sasank Chilamkurthy

# material from notebook at: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '../../../stanford_dogs_new/'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                              shuffle=True, num_workers=4)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated
        
# # Get a batch of training data
# inputs, classes = next(iter(dataloaders['train']))
# 
# # Make a grid from batch
# out = torchvision.utils.make_grid(inputs)
# 
# imshow(out, title=[class_names[x] for x in classes])

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()
    
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])
                
                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [3]:
model_vgg16 = models.vgg16(pretrained=True)

for param in model_vgg16.parameters():
    param.requires_grad = False

print(model_vgg16)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/rka73/.cache/torch/hub/checkpoints/vgg16-397923af.pth



VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=

In [4]:
# ref: https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

model_vgg16.classifier = nn.Sequential(*list(model_vgg16.classifier.children())[:-1] + [nn.Linear(in_features=4096, out_features=120, bias=True)])

model_vgg16 = model_vgg16.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_vgg16 = optim.SGD(model_vgg16.classifier.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_vgg16, step_size=7, gamma=0.1)

In [5]:
model_vgg16 = train_model(model_vgg16, criterion, optimizer_vgg16, exp_lr_scheduler, num_epochs=30)




Epoch 0/29
----------
train Loss: 1.8465 Acc: 0.5568
val Loss: 0.7438 Acc: 0.8075

Epoch 1/29
----------
train Loss: 1.7428 Acc: 0.6143
val Loss: 0.8626 Acc: 0.8056

Epoch 2/29
----------
train Loss: 1.8193 Acc: 0.6239
val Loss: 0.7680 Acc: 0.8163

Epoch 3/29
----------
train Loss: 1.8340 Acc: 0.6335
val Loss: 0.9268 Acc: 0.8241

Epoch 4/29
----------
train Loss: 1.8743 Acc: 0.6402
val Loss: 0.8416 Acc: 0.8304

Epoch 5/29
----------
train Loss: 1.8351 Acc: 0.6479
val Loss: 0.8834 Acc: 0.8309

Epoch 6/29
----------
train Loss: 1.8775 Acc: 0.6500
val Loss: 0.8401 Acc: 0.8309

Epoch 7/29
----------
train Loss: 1.6284 Acc: 0.6796
val Loss: 0.6941 Acc: 0.8504

Epoch 8/29
----------
train Loss: 1.5689 Acc: 0.6849
val Loss: 0.6719 Acc: 0.8558

Epoch 9/29
----------
train Loss: 1.4733 Acc: 0.6981
val Loss: 0.6595 Acc: 0.8596

Epoch 10/29
----------
train Loss: 1.4531 Acc: 0.6937
val Loss: 0.6509 Acc: 0.8592

Epoch 11/29
----------
train Loss: 1.4168 Acc: 0.6970
val Loss: 0.6406 Acc: 0.8558

Ep

In [6]:
torch.save(model_vgg16.state_dict(), 'vgg16_based_model2.pt')

## Load the Model and test it

In [9]:
model_vgg16 = models.vgg16(pretrained=True)

for param in model_vgg16.parameters():
    param.requires_grad = False

model_vgg16.classifier = nn.Sequential(*list(model_vgg16.classifier.children())[:-1] + [nn.Linear(in_features=4096, out_features=120, bias=True)])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_vgg16 = model_vgg16.to(device)



model_vgg16.load_state_dict(torch.load('vgg16_based_model2.pt'), strict=False)
model_vgg16.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## Train the VGG16 model - 30 ep with Adamax optimizer

In [12]:
# ref: https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

vgg16 = models.vgg16(pretrained=True)

for param in vgg16.parameters():
    param.requires_grad = False
    
vgg16.classifier = nn.Sequential(*list(vgg16.classifier.children())[:-1] + [nn.Linear(in_features=4096, out_features=120, bias=True)])

vgg16 = vgg16.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_vgg16_adamax = optim.Adamax(vgg16.parameters())

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_vgg16, step_size=7, gamma=0.1)

In [13]:
vgg16_adamax_30ep = train_model(vgg16, criterion, optimizer_vgg16_adamax, exp_lr_scheduler, num_epochs=30)

# %

Epoch 0/29
----------
train Loss: 1.5202 Acc: 0.5917


/home/rka73/.local/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


val Loss: 0.5235 Acc: 0.8358

Epoch 1/29
----------
train Loss: 1.1215 Acc: 0.6844
val Loss: 0.4848 Acc: 0.8470

Epoch 2/29
----------
train Loss: 1.1095 Acc: 0.6962
val Loss: 0.4868 Acc: 0.8465

Epoch 3/29
----------
train Loss: 1.1016 Acc: 0.7052
val Loss: 0.4912 Acc: 0.8484

Epoch 4/29
----------
train Loss: 1.1015 Acc: 0.7085
val Loss: 0.4946 Acc: 0.8548

Epoch 5/29
----------
train Loss: 1.0947 Acc: 0.7172
val Loss: 0.4854 Acc: 0.8562

Epoch 6/29
----------
train Loss: 1.1207 Acc: 0.7152
val Loss: 0.4980 Acc: 0.8519

Epoch 7/29
----------
train Loss: 1.0858 Acc: 0.7273
val Loss: 0.5056 Acc: 0.8514

Epoch 8/29
----------
train Loss: 1.0944 Acc: 0.7254
val Loss: 0.4919 Acc: 0.8621

Epoch 9/29
----------
train Loss: 1.1014 Acc: 0.7275
val Loss: 0.5030 Acc: 0.8553

Epoch 10/29
----------
train Loss: 1.0935 Acc: 0.7345
val Loss: 0.5020 Acc: 0.8635

Epoch 11/29
----------
train Loss: 1.1365 Acc: 0.7280
val Loss: 0.5203 Acc: 0.8611

Epoch 12/29
----------
train Loss: 1.1351 Acc: 0.7271
v

In [14]:
torch.save(vgg16_adamax_30ep.state_dict(), 'vgg16_adamax_30ep_based_model2.pt')

## Load the Adamax optimizer based vgg16 model and test it

In [16]:
vgg16_adamax_30ep_loaded = models.vgg16(pretrained=True)

for param in vgg16_adamax_30ep_loaded.parameters():
    param.requires_grad = False

vgg16_adamax_30ep_loaded.classifier = nn.Linear(in_features=4096, out_features=120, bias=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16_adamax_30ep_loaded = vgg16_adamax_30ep_loaded.to(device)



vgg16_adamax_30ep_loaded.load_state_dict(torch.load('vgg16_adamax_30ep_based_model2.pt'), strict=False)
vgg16_adamax_30ep_loaded.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1